https://craffel.github.io/mir_eval/#module-mir_eval.transcription

## Imports

In [8]:
import os
import numpy as np
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import SGD
from mir_eval.transcription import precision_recall_f1_overlap as eval 

## Constants

In [9]:
BASELINE_TEST_DIR = '../results/baseline_test/'
BASELINE_ARCH = os.path.join(BASELINE_TEST_DIR, 'baseline_test.json')
BASELINE_WEIGHTS = os.path.join(BASELINE_TEST_DIR, 'baseline_test.h5')
DATASET_DIR = '../splits/config-2_logfilt_baseline/'
TEST_SET_DIR = os.path.join(DATASET_DIR, 'test')

## Load model

In [10]:
# Load architecture
json_file = open(BASELINE_ARCH)
loaded_baseline_json = json_file.read()
json_file.close()

# Load weights
baseline = model_from_json(loaded_baseline_json)
baseline.load_weights(BASELINE_WEIGHTS)
baseline.compile(
    loss='binary_crossentropy',
    optimizer=SGD(lr=0.1, momentum=0.9),
    metrics=['accuracy', 'mse', 'mae'])

## Load datapoints 

In [11]:
def read_mm(data_dir, type, ID):
    NOTE_RANGE = 88
    WINDOW_SIZE = 5
    N_BINS = 229

    input_path = os.path.join(data_dir, type, ID)
    output_path = os.path.join(data_dir, 'expect', ID)

    mm_input = np.memmap(input_path, mode='r')
    mm_output = np.memmap(output_path, mode='r')
    input = np.reshape(mm_input, (-1, WINDOW_SIZE, N_BINS))
    output = np.reshape(mm_output, (-1, NOTE_RANGE))

    return input, output

In [12]:
test_datapoints = os.listdir(TEST_SET_DIR)

In [13]:
X, y = [], []
for track_name in test_datapoints:
    input, output = read_mm(DATASET_DIR, 'test', track_name)
    
    X.append(input)
    y.append(output)

X = np.concatenate(X)
y = np.concatenate(y)


In [15]:
print X.shape
print y.shape

(38901, 5, 229)
(38901, 88)


In [16]:
X = X[2000:4000]

## Evaluate with Kelz Func

In [17]:
predictions = baseline.predict(X, verbose=1)

2000/2000 [==============================] - 30s 15ms/step


In [18]:
print predictions.shape
print y.shape

(2000, 88)
(38901, 88)


In [19]:
def eval_framewise(predicts, targets, thresh=0.5):
    """
     author: filip (+ data-format amendments by rainer)
     """
    if predicts.shape != targets.shape:
         raise ValueError('predictions.shape {} != targets.shape {}'.format(predictions.shape, targets.shape))

    pred = predicts > thresh
    targ = targets > thresh

    tp = pred & targ
    fp = pred ^ tp
    fn = targ ^ tp

    # tp, fp, tn, fn
    return tp.sum(), fp.sum(), 0, fn.sum()

In [20]:
tp_total, fp_total, fn_total = 0, 0, 0
for p, t in zip(predictions, y[2000:4000]):
    tp, fp, _, fn = eval_framewise(p, t)
    tp_total += tp
    fp_total += fp
    fn_total += fn
    
tp_total, fp_total, fn_total

(0, 0, 0)

## Evaluate with mir_eval

TO DO